<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/2_Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Będzie to funkcja która pozwoli wczytać dane z pliku wav, podzieli je na M-ms klipy (500 ms domyślne) z N-sek overlapem (150 ms), wczyta etykiety z pliku txt (jeśli więcej niż 4ms głosu w próbce - to pozytywna), stworzy wektory 

* X_train i Y_train
* X_valid i Y_valid

W sumie mam 94 nagrania. 

* treningowy - 87 nagrań, balansujemy, jeśli są dostępne inne etykiety niż interesujących nas głosów (np gh, t, g), to dodajemy je jako próbki negatywne. Jeśłi nie ma głosów w nagraniu to bierzemy P (50) próbek losowych z tego nagrania. 
* walidacyjny to 7-8 nagrań, które balansujemy tylko trochę - np z poziomu 1:100, do poziomu 1:10.




In [3]:
# Wczytywanie google drive'a
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function

Mounted at /content/drive


In [4]:
import os
import contextlib
import numpy as np
import wave
import math
import pandas as pd

In [5]:
# Parametry które będą wchodzić do funkcji

path_test1618_txt = 'drive/My Drive/poprawione etykiety 24112020/testowe/'
path_train161718_txt =  'drive/My Drive/poprawione etykiety 24112020/treningowe/'
path_test1618_wav = 'drive/My Drive/testowe_1618/'
path_train161718_wav =  'drive/My Drive/treningowe94_161718/'
balance_type = ['balanced', 'valid', 'full_rec']  # na potrzeby treningu, valida, testu
chunk_length_ms = 500 # 500 ms
chunk_overlap = 150 # 150 ms
calls_0 = ['t', 'g', 'czapla', 'gh', 'puszczyk']                                    # etykiety które są negatywne
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni'] # calls of interest
calls_unknown = ['???','??? mysz', '??? high freq']                                 # unknown, nie wiem czy to głos czy nie, więc będę wyrzucać te chunksy to zawierające
tolerance = 0.004  # jaka tolerancja, w sek (jeśli mniej niż ta wartość znajdzie się w chunksie, to uznajemy że głosu tam nie ma)

In [6]:
def my_read_labels(label_name,path_txt):  # ta sama funkcja co w 1. poprawianie danych
    '''Wczytuje etykiety czasowe z pliku labels.txt w folderze train.
    
    Returns:
        ndarray: Tablica z etykietami czasowymi zawierająca kolumny:  sekunda początku dźwięku, sekunda końca dźwięku, gatunek.
    '''
    labels = []
    with open(os.path.join(path_txt, label_name + '.txt'), 'r') as file:
        text = file.read()
        for line in text.split('\n')[0:]:
            if len(line) > 1:
                start, stop, spec = line.split('\t')
                #print(start)
                start, stop, spec = float(start), float(stop), str(spec),
                labels.append([start, stop, spec])
    return np.array(labels)


def my_check_labels(second, chunk_length_s, labels, tol=0.):
    '''Sprawdza czy w ramce czasowej [second, second + chunk_length_s] znajduje się coś wg etykiet `labels`.
    
    Args:
        second (float): Sekunda nagrania.
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a 2ga - koniec nagrania.
        tol (float): Tolerancja na brzegach fragmentu. Dźwięk, żeby był uznany, musi się kończyć po czasie `second+tol`
            lub zaczynać przed czasem `second+chunk_length-tol`.
    Returns:
        bool: Czy w ramce czasowej jest etykieta - co najmniej 4 ms labelu.
 ''' 
    
  #calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']

  # jeśli początek etykiety zawiera się w chunku, najpóźniej 4ms przed końcem chunka
  # jeśli koniec etykiety zawiera się w chunku, i najwcześniej 4 ms po rozpoczęciu chunka
  # jeśli początek etykiety zaczyna się rpzed, a koniec po (w przypadku głosów dłuższych niż chunk length, b. rzadkie przypadki dla 0.5 sek)
    return (float(labels[0]) >= second and float(labels[0]) < second + chunk_length_s - tol) or \
           (float(labels[1]) < second + chunk_length_s and float(labels[1]) > second + tol) or \
           (float(labels[0]) < second and float(labels[1]) > second + chunk_length_s)              
           #and (labels[2] in calls_to_cut)    # nie sprawdzajmy tu jaka to etykieta jest

def my_map_seconds_to_y(labels, recording_duration, calls_of_interest, calls_to_cut, calls_unknown):  ## Chcemy 500 ms z 150 ms overlapem
    '''Tworzy etykiety dla każdego kolejnego chunksa. 1 gdy co najmniej 4 ms etykiety się znajdują w chunksie (gdy urwane dźwięki na brzegach, <4ms - to 0).
    
    Args:
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a druga - koniec nagrania.
    Returns:
        ndarray: Tablica z binarnymi etykietami.
    '''
    #STARE: calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
    
    
    duration_in_ms = recording_duration*1000
    nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
    print("ilość chunksów w tym nagraniu: ", math.ceil(nr_of_chunks) )
    
    
    y = [0] * math.ceil(nr_of_chunks)             # recording_duration 
    y_restrictive = [0] * math.ceil(nr_of_chunks) # recording_duration
    chunks_start = [0] * math.ceil(nr_of_chunks)     
    chunks_end = [0] * math.ceil(nr_of_chunks) 
    has_unknown = [0] * math.ceil(nr_of_chunks)
    call_id = [[] for _ in range(math.ceil(nr_of_chunks))]
    chunks_species = [[] for _ in range(math.ceil(nr_of_chunks))]
    
    print(np.shape(labels))
    for s in range(math.ceil(nr_of_chunks)):
        chunks_start[s] = s * ((chunk_length_ms-chunk_overlap) / 1000)  # czyli s * 0.35
        chunks_end[s] = chunks_start[s] + chunk_length_ms/1000 
        for ind,l in enumerate(labels):
            #print('label_index: ',ind, ' a label to: ', l[2] )
            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l):
                #y[s] = 1
                #print(s*0.35)
                if l[2] in calls_to_cut:
                  y[s] = 0
                if l[2] in calls_unknown:
                  y[s] = 0 
                  has_unknown[s] = 1
                if l[2] in calls_of_interest:  # ostatnie, bo jeśli jest więcej niż 1 etykieta, to najważniejsze jest że w chunksie jest też głos
                  y[s] = 1 
                    
                  #chunks_species[s]= l[2]  
                  #call_id[s] = ind[2] 

            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l, 0.004): #z tolerancją 
                  #y_restrictive[s] = 0
                  #print("wyciety glos:")
                  #print(l[2])
                if l[2] in calls_to_cut:
                  y_restrictive[s] = 0
                if l[2] in calls_unknown:
                  y_restrictive[s] = 0  
                  has_unknown[s] = 1
                if l[2] in calls_of_interest:  # ostatnie, bo jeśli jest więcej niż 1 etykieta, to najważniejsze jest że w chunksie jest też głos
                  
                  #if y_restrictive[s] == 0:  # pierwszy label 
                  y_restrictive[s] = 1  
                  chunks_species[s].append(l[2])   # może być w tym, nie w restrictive - by było wiadomo jaki typ głosu zostanie wycięty
                  call_id[s].append(ind)
              
                     



                 
        if y[s] != y_restrictive[s] and l[2] in calls_of_interest:
            print('próbka ', s, 'zaczynajaca sie ', chunks_start[s],', jest unknown, bo za mały fragment głosu ')
            #print('y[s]:', y[s],'yres:', y_restrictive[s] )
            y[s] = 0    # jeśli mniej niż 4 ms fragment, to zakładamy że nie ma głosu, i dajemy unknown także, by nie karać za ewentualne znalezienie tego głosu
            has_unknown[s] = 1
    print('Rozmiar ',np.shape([s, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown]))
    return s+1, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown    


def my_load_wav(path_wav, path_txt):   # dłuższa funkcja niż w 1.

  recording_labels_all = []
  in_which_recording = []
  file_names = []

  rec_files = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]

  print(np.size(rec_files))

  X_matrix = [[] for _ in range(np.size(rec_files))]
  file_names = [[] for _ in range(np.size(rec_files))]

  for ind, file_name in enumerate(rec_files):
      
      print("------------Analiza nagrania: " + file_name + "-----------")
      recording_id = str(file_name.split('.')[0])
      
      fname = path_wav + file_name      #print(fname)
      with contextlib.closing(wave.open(fname,'r')) as f:
          frames = f.getnframes()
          rate = f.getframerate()
          duration = frames / float(rate)
          recording_duration = math.ceil(duration)
          print("Czas trwania nagrania - w sekundach: " + str(recording_duration))
          #print("rate:", rate)
    
      recording_id = (file_name.split('.')[0])       #print(recording_id)
      recording_labels = my_read_labels(recording_id, path_txt)    

      print("Duration:", duration)
      #y_binary = my_map_seconds_to_y(recording_labels, duration)
      chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown  = my_map_seconds_to_y(recording_labels, duration, calls_1, calls_0, calls_unknown)
      chunk_ids = range(chunk_id)
      chunk_start = rate*np.array(chunk_start)
      chunk_start = [round(num) for num in chunk_start]
      chunk_end = rate*np.array(chunk_end)
      chunk_end = [round(num) for num in chunk_end]
      X_matrix[ind] = [chunk_ids, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown]  # !!!! chunk start i end  przerzucone na sample 1!!!!!
      file_names[ind] = file_name

  return X_matrix , file_names #chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown

In [7]:
#chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown = my_load_wav(path_train161718_wav, path_train161718_txt) 

X_train, file_names = my_load_wav(path_test1618_wav, path_test1618_txt) 



20
------------Analiza nagrania: BUK4_20160922_005604.wav-----------
Czas trwania nagrania - w sekundach: 1799
Duration: 1798.002358276644
ilość chunksów w tym nagraniu:  5137
(40, 3)
próbka  1461 zaczynajaca sie  511.34999999999997 , jest unknown, bo za mały fragment głosu 
Rozmiar  (7,)
------------Analiza nagrania: BUK4_20161016_035704.wav-----------
Czas trwania nagrania - w sekundach: 1799
Duration: 1798.002358276644
ilość chunksów w tym nagraniu:  5137
(28, 3)
Rozmiar  (7,)
------------Analiza nagrania: BUK5_20161101_002104b.wav-----------
Czas trwania nagrania - w sekundach: 1799
Duration: 1798.002358276644
ilość chunksów w tym nagraniu:  5137
(273, 3)
próbka  436 zaczynajaca sie  152.6 , jest unknown, bo za mały fragment głosu 
próbka  861 zaczynajaca sie  301.34999999999997 , jest unknown, bo za mały fragment głosu 
próbka  972 zaczynajaca sie  340.2 , jest unknown, bo za mały fragment głosu 
próbka  1075 zaczynajaca sie  376.25 , jest unknown, bo za mały fragment głosu 
próbk

In [8]:
print(np.shape(X_train))
czas_trwania_spr= np.array(X_train[0][1])- np.array(X_train[0][2])
print(np.unique(czas_trwania_spr)) # upewnienie się że wszystkie chunki jednakowej długości są
print(np.shape(file_names))

result_dataframe = pd.DataFrame(data = X_train, index=file_names, columns = ['chunk_ids', 'chunk_start', 'chunk_end', 'has_bird', 'chunks_species', 'call_id', 'has_unknown'])

(20, 7)
[-22050.]
(20,)


In [ ]:
#@title
a = (X_train[17][1])
print(type(X_train))
print(type(result_dataframe['chunk_start']))
print(result_dataframe['chunk_start']["BUK4_20160922_005604.wav"])

In [9]:
# do sprawdzenia czy te zagnieżdżone listy w call_id i chunks_species działają

def isListEmpty(inList):
  for elem in inList:
    if elem: # Is a list
        print(elem)
        #return all( map(isListEmpty, inList) )
    #return False # Not a list

isListEmpty(result_dataframe['call_id'][2])
isListEmpty(result_dataframe['chunks_species'][2])


[4]
[4]
[5]
[5]
[6, 7]
[6, 7, 8, 9]
[8, 9]
[11]
[11]
[12]
[16]
[16, 17]
[17]
[19]
[19, 20]
[20]
[21]
[22]
[22]
[23]
[24]
[24]
[25]
[25]
[26]
[26]
[27]
[27, 28]
[28]
[29, 30]
[30]
[31]
[31, 32]
[33]
[34]
[34]
[35]
[36]
[36]
[37]
[37, 38]
[38]
[39]
[39, 40]
[40]
[41]
[42, 43]
[42, 43]
[44]
[44, 45]
[45, 46]
[46]
[48]
[48]
[48, 49]
[49]
[50]
[50]
[51]
[52]
[52]
[53]
[53, 54]
[54, 55]
[55, 56]
[57]
[57]
[58]
[59]
[61]
[61]
[62]
[62, 63]
[63]
[64]
[64, 65, 66]
[65, 66]
[66]
[67]
[67]
[68]
[68]
[69]
[70]
[70]
[70, 71]
[71]
[71]
[71]
[72]
[72]
[73]
[73, 74]
[74]
[75]
[75]
[75]
[76]
[76, 77]
[77]
[78]
[79]
[79]
[80]
[80]
[81]
[81]
[82]
[82]
[83]
[83]
[84]
[84]
[84]
[85]
[85]
[86]
[86, 87]
[87]
[88]
[88, 89]
[89]
[92]
[94]
[94]
[95, 96]
[96, 97]
[97]
[98]
[98]
[99]
[99, 100, 101]
[101]
[102]
[102]
[103]
[103, 104]
[104]
[105]
[105]
[106]
[106, 107]
[107, 108]
[108]
[109]
[109, 110]
[110, 111]
[111]
[112]
[112]
[113]
[113, 114]
[114]
[115]
[115]
[116]
[116]
[117]
[117]
[118]
[118]
[119]
[120]
[1

In [10]:
# do sprawdzenia jakie ID mają głosy w naszym nagraniu (każdy z labeli ma ID, nie tylko głosy, dlatego nie wszystkie liczby tu będą)

test_list = result_dataframe['call_id'][2]

#print("The original list : " + str(test_list)) 
  
res = [] 
temp = set() 
for inner in test_list: 
        for ele in inner: 
            if not ele in temp: 
                temp.add(ele) 
                res.append(ele) 
  
print("Unique elements in nested tuples are : " + str(res)) 

Unique elements in nested tuples are : [4, 5, 6, 7, 8, 9, 11, 12, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 22